In [49]:
# Author: @jacoblarget, with advice from Prof. Shreya Kumar
# Last Updated: 28 April 2020

# import relevant packages
import matplotlib.pyplot as plt # for visualization
import numpy as np # for trignometric functions and constants
from ipywidgets import interact # for interactive display

# print preface to user
print('''The following project simulates air cannon projectile motion.
It is prepared to launch a tennis ball on Earth out of a standard 4-liter air tank.
You get to choose the values for 5 parameters.
(1) Pressure of the tank when launch begins (in kiloPascals)
(2) Drag of the air on the ball (in proportional strength units)
(3) Launch angle of the cannon (in degrees)
(4) Cannon barrel length (in meters)
(5) Elevation of cannon above the surface (in meters)
You'll get back a number of statistics about the launch, as well as a visualization of the projectile path.
The cannon is represented by a black line, ad the red dot-dashed line is the projectile.
Feel free to play around with the different sliders to learn valuable insights about projectile motion.
''')

# initialize relevant constants
SIZE = 4 # L, volume of a standard tank
GRAVITY = 9.81 # m/s^2, acceleration due to Earth's gravity
DENSITY = 1.225 # kg/m^3, density of air on Earth
AREA = 0.0035 # m^2, cross sectional area of tennis ball
MASS = 0.0585 # kg, mass of tennis ball
FRICTION = 0.5 # unitless coefficient of friction against tube surface for tennis ball
TIMESTEP = .01 # arbitrary level of discretization, decrease value to increase computation time and accuracy

# create visualization dynamically through function call
def interactive_plot(pressure, drag, angle, barrel, elevation):
  # convert user's degrees into calculatable radians
  angle *= np.pi/180

  # create cannon graphic
  x_cannon_position = [0,barrel*np.cos(angle)]
  y_cannon_position = [elevation, barrel*np.sin(angle) + elevation]
  plt.plot(x_cannon_position,y_cannon_position, "black")

  # initialize lists, counter for timestepping inside cannon
  cannon_position = [0]
  cannon_velocity = [0]
  cannon_acceleration = [pressure*AREA/MASS
                         - GRAVITY*FRICTION*(np.cos(angle)+np.sin(angle))]
  cannon_count = 0

  # timestep with physics equations to simulate the inside of an air cannon during launch
  # ends when the position of the cannon is past the length of the barrel.

  while(cannon_position[cannon_count] < barrel):
    cannon_position.append(cannon_position[cannon_count]
                           + cannon_velocity[cannon_count]*TIMESTEP
                           + cannon_acceleration[cannon_count]/2*(TIMESTEP**2)
                           )
    cannon_velocity.append(cannon_velocity[cannon_count]
                           + cannon_acceleration[cannon_count]*TIMESTEP
                           )
    cannon_acceleration.append(pressure*SIZE*AREA/MASS/(AREA*cannon_position[cannon_count + 1] + SIZE)
                               - DENSITY*drag*AREA*(cannon_velocity[cannon_count + 1]**2)/2/MASS
                               - GRAVITY*FRICTION*(np.cos(angle)+np.sin(angle)) 
                              )
    cannon_count += 1

  # initialize lists, counter for timestepping outside cannon
  x_air_position = [x_cannon_position[-1]]
  y_air_position = [y_cannon_position[-1]]
  x_air_velocity = [cannon_velocity[cannon_count]*np.cos(angle)]
  y_air_velocity = [cannon_velocity[cannon_count]*np.sin(angle)]
  x_air_acceleration = [-DENSITY*drag*AREA/2/MASS*(cannon_velocity[cannon_count]**2)*np.cos(angle)]
  y_air_acceleration = [-DENSITY*drag*AREA/2/MASS*(cannon_velocity[cannon_count]**2)*np.sin(angle) - GRAVITY]
  air_count = 0

  # timestep with physics equations to simulate the outside of an air cannon during launch
  # ends with vertical position of the projectile hits the ground.

  while(y_air_position[air_count] >= 0):
    x_air_position.append(x_air_position[air_count]
                          + x_air_velocity[air_count]*TIMESTEP
                          + x_air_acceleration[air_count]/2*(TIMESTEP**2)
                          )
    y_air_position.append(y_air_position[air_count]
                          + y_air_velocity[air_count]*TIMESTEP
                          + y_air_acceleration[air_count]/2*(TIMESTEP**2)
                          )
    x_air_velocity.append(x_air_velocity[air_count]
                          + x_air_acceleration[air_count]*TIMESTEP
                          )
    y_air_velocity.append(y_air_velocity[air_count]
                          + y_air_acceleration[air_count]*TIMESTEP
                          )
    x_air_acceleration.append(-DENSITY*drag*AREA/2/MASS
                              * (y_air_velocity[air_count+1]**2 + x_air_velocity[air_count + 1]**2)**0.5
                              * x_air_velocity[air_count + 1]
                              )
    y_air_acceleration.append(-DENSITY*drag*AREA/2/MASS
                          * (y_air_velocity[air_count+1]**2 + x_air_velocity[air_count + 1]**2)**0.5
                          * y_air_velocity[air_count + 1]
                          - GRAVITY
                          )
    air_count += 1
    
    # preface to visualization and creation
  print('''The axes have a 1:1 fixed scale so you can see their relative performance. How close can you get to the edges? (Maximum Height: 58.93, Maximum Range: 44.08)
Note: It may take a few moments for the visualization to readjust.''')
  # Maximum Height Settings(900,0,85,1.5,50)
  # Maximum Range Settings(900,0,15,1.5,50)
  plt.plot(x_air_position[:-1], y_air_position[:-1], 'r-.') # ignore the last point since it went through the ground
  plt.axis([0,60, 0, 60]) # these values are based off of the maximum height this simulation can achieve, and is square to preserve scaling
  plt.title('Projectile Path')
  plt.xlabel('Distance away from the cannon(m)')
  plt.ylabel('Distance above ground elevation (m)')
  plt.show()

  # inform user of launch statistics
  print('''
    Here are some interesting launch statistics.
    Initial cannon acceleration achieved in cannon (m/s^2): {:.2f}
    Time in cannon (s): {:.2f}
    Horizontal and vertical components of exit velocity (m/s): ({:.2f}, {:.2f})
    Time of flight (s): {:.2f}
    Maximum Height achieved by flight (m): {:.2f}
    Maximum Range achieved by flight (m): {:.2f}'''.format(cannon_acceleration[0],
                                                           cannon_count*TIMESTEP,
                                                           cannon_velocity[cannon_count]*np.cos(angle),
                                                           cannon_velocity[cannon_count]*np.sin(angle),
                                                           air_count*TIMESTEP,
                                                           max(y_air_position),
                                                           max(x_air_position))
  )
  
# run function with interact framework
interact(interactive_plot,
          pressure = (150, 900, 50),
          drag = (0, 0.9, 0.1),
          angle = (5, 85, 5),
          barrel = (0.1, 1.55, 0.1),
          elevation = (0, 50, 1)
         )
plt.show() # this cleans the bottom of the output window

The following project simulates air cannon projectile motion.
It is prepared to launch a tennis ball on Earth out of a standard 4-liter air tank.
You get to choose the values for 5 parameters.
(1) Pressure of the tank when launch begins (in kiloPascals)
(2) Drag of the air on the ball (in proportional strength units)
(3) Launch angle of the cannon (in degrees)
(4) Cannon barrel length (in meters)
(5) Elevation of cannon above the surface (in meters)
You'll get back a number of statistics about the launch, as well as a visualization of the projectile path.
The cannon is represented by a black line, ad the red dot-dashed line is the projectile.
Feel free to play around with the different sliders to learn valuable insights about projectile motion.



interactive(children=(IntSlider(value=500, description='pressure', max=900, min=150, step=50), FloatSlider(val…